In [1]:
import pandas as pd
import os
from os.path import join  
import numpy as np
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


In [2]:
prelim_dir = '/Users/atecza/Library/CloudStorage/OneDrive-RMI/Documents/OCI Phase 2/Midstream/PRELIM_v1.6.xlsm'

In [3]:
sp_dir = '/Users/atecza/Library/CloudStorage/OneDrive-RMI/Documents/OCI Phase 2'

In [4]:

opgee_path = '/Users/atecza/Library/CloudStorage/OneDrive-RMI/Documents/ClimateTRACE Phase 4/csv/db_exports/trace_main_inputs.csv'

opgee_df = pd.read_csv(opgee_path)


/var/folders/dl/sbgvnlps4h5_kfklrckvd58h0000gp/T/ipykernel_97649/1728311615.py:3: DtypeWarning: Columns (122,123,135,144) have mixed types. Specify dtype option on import or set low_memory=False.
  opgee_df = pd.read_csv(opgee_path)


### Extract Assay Information

In [5]:
def extract_assay_info(file):
    
    df = pd.read_excel(file,sheet_name='Assay Inventory', na_values=['NA'], usecols="B:E",skiprows=7, names=['Country', 'Selected Assay', 'Units', 'Full Crude'])
    
    country_list = []
    region_list = []
    assay_names = []
    assay_numbers = []
    full_crude_vol_flows = []
    full_crude_sulphurs = []
    full_crude_apis = []

    for i in df.index:
        
        country_region = df.at[i,'Country']
        if pd.notnull(country_region):
            
            cr_list = country_region.split('-')
            
            if len(cr_list) > 1:
                
                country = cr_list[0]
                region = cr_list[1]
                
                country_list.append(country)
                region_list.append(region)
            
            else:
                country_list.append(country_region)
                region_list.append(np.nan)
                
            assay_names.append(df.at[i,'Selected Assay'])
            assay_numbers.append(df.at[i+1,'Units'])

            #get vol flow
            full_crude_vol_flows.append(df.at[i+3,'Full Crude'])

            #get sulpher
            full_crude_sulphurs.append(df.at[i+6,'Full Crude'])

            #get api
            full_crude_apis.append(df.at[i+8,'Full Crude'])



    #df_assay = pd.DataFrame(list(zip(country_list, region_list, assay_names, assay_numbers, units_flow, full_crude_vol_flows, units_sulphur, full_crude_sulphurs,units_api,full_crude_apis)), columns=['Country','Region','Assay Name','Assay Number','Throughput Unit','Throughput','Sulphur Unit','Sulphur','API Unit','API'])
    df_assay = pd.DataFrame(list(zip(country_list, region_list, assay_names, assay_numbers, full_crude_vol_flows, full_crude_sulphurs,full_crude_apis)), columns=['Country','Region','Assay Name','Assay Number','Throughput','Sulphur','API'])
    
    return df_assay
        

In [6]:
assay_df = extract_assay_info(prelim_dir)

#### Separate Assay Name from Year

In [7]:

split_assay_year = [int(re.findall(r'\b\d+\b',x)[0]) if len(re.findall(r'\b\d+\b',x))>0 else np.nan for x in assay_df['Assay Name']]
assay_df['Year'] = [int(x) if x > 1000 else np.nan for x in split_assay_year]
assay_df['Year'] = assay_df['Year'].astype('Int64')
assay_df['Assay Name NN'] = [" ".join(x.split()[:-1]) if x.split()[-1].isnumeric() else x for x in assay_df['Assay Name']]

In [8]:
assay_df['Country'].replace('US','United States', inplace=True)

In [9]:
assay_df['Country'].replace('Sudan','South Sudan', inplace=True) 

In [10]:
assay_df['Country'].replace('Russia','Russian Federation', inplace=True)

In [11]:
assay_df['Country'].replace('Vietnam','Viet Nam', inplace=True)
assay_df['Country'].replace('UK','United Kingdom', inplace=True)

assay_df['Country'].replace("Cote d' Ivoire",'Cote dIvoire', inplace=True)
assay_df['Country'].replace('Republic of Congo','Congo', inplace=True)
assay_df['Country'].replace('Papau New Guinea','Papua New Guinea', inplace=True)
assay_df['Country'].replace('Timor Leste','Timor-Leste', inplace=True)
assay_df['Country'].replace('Congo Republic','Congo', inplace=True)

In [12]:
assay_df.head(2)

,Country,Region,Assay Name,Assay Number,Throughput,Sulphur,API,Year,Assay Name NN
0,Nigeria,NaN,Abo_Solomon 2010,11,100032.667139,0.147911,36.396513,2010,Abo_Solomon
1,Canada,Alberta,Access Western Blend_Crude Monitor,NaN,99851.088615,4.196450,22.640000,<NA>,Access Western Blend_Crude Monitor


### Import Field Data

In [13]:
opgee_df = opgee_df.loc[:,['Field_name','Field_location_Country','Region','US_state','API_gravity']]

In [14]:
field_names_df = opgee_df.drop_duplicates(subset=['Field_name']).reset_index(drop=True)

In [15]:
RMI_col = []
Assay_col = []

missing_list = ['Belgium', 'Bangladesh', 'Uzbekistan', 'France', 'Ireland',
       'Chile', 'Ukraine', 'South Korea', 'Serbia', 'Israel',
       'The Democratic Republic of the Congo', 'Barbados', 'Bolivia',
       'Kyrgyzstan', 'Japan', 'Tanzania', 'Croatia', 'South Africa',
       'Netherlands', 'Estonia', 'Mozambique', 'Slovenia', 'Greece',
       'Senegal', 'Bulgaria', 'Morocco', 'Jordan', 'Georgia', 'Cuba',
       'Benin', 'Myanmar', 'Belize', 'Suriname', 'Spain']

for CN in missing_list:
    fuzz_scores = process.extract(CN, assay_df['Country'], scorer=fuzz.token_set_ratio)
    fuzz_high_scores = [x[0] for x in fuzz_scores if x[1] > 80]
    if len(fuzz_high_scores) > 0:
        Assay_col.append(fuzz_high_scores[0])
        RMI_col.append(CN)
        

check_df = pd.DataFrame(list(zip(RMI_col, Assay_col)))


In [16]:
check_df

,0,1
0,The Democratic Republic of the Congo,Congo


### Match Assay to Field Name

Will capture when multiple assays match a single field, will also capture when multiple fields match the same assay

In [17]:
rmi_name_list = []
assay_matches = []
fuzzy_check = []

for i in field_names_df.index:
    FN = field_names_df.at[i,'Field_name']
    assay_names = assay_df['Assay Name NN'] #grab all the possible assay names
    match_list = [x for x in assay_names if FN in x]
    
    #If more than one assay matches
    if len(match_list) > 0:
        
        rmi_name_list.append(FN)
        assay_matches.append(match_list)
        fuzzy_check.append(0)
        
    if len(match_list) == 0:
        
        fuzz_scores = process.extract(FN, assay_names, scorer=fuzz.token_set_ratio)
    
        fuzz_high_scores = [x[0] for x in fuzz_scores if x[1] > 70]
        
        if len(fuzz_high_scores) > 0:
            
            rmi_name_list.append(FN)
            assay_matches.append(fuzz_high_scores)
            fuzzy_check.append(1)
    
        #If there are no exact matches and no fuzzy matches
        else:
            
            if field_names_df.at[i,'Field_location_Country'] == 'United States':
                State = field_names_df.at[i,'US_state']
                state_match = assay_df[assay_df['Region']==State]['Assay Name NN'].values
                
                if len(state_match)>0:
                    rmi_name_list.append(FN)
                    assay_matches.append(state_match)
                    fuzzy_check.append(0)
                
                #If no state match in US
                else:
                    country_match = assay_df[assay_df['Country']=='United States']['Assay Name NN'].values
                    
                    rmi_name_list.append(FN)
                    assay_matches.append(country_match)
                    fuzzy_check.append(0)
                    
                    
            #If not in the US       
            else:
                CN = field_names_df.at[i,'Field_location_Country']
                country_match = assay_df[assay_df['Country']==CN]['Assay Name NN'].values
                
                rmi_name_list.append(FN)
                assay_matches.append(country_match)
                fuzzy_check.append(0)
                
               
                
   
    df_match = pd.DataFrame(list(zip(rmi_name_list, assay_matches, fuzzy_check)), columns=['RMI Name','Assay Match','Fuzzy'])


### Unpack the list of assay names

In [18]:
rmi_name = []
assay_match_name =[]
for i in df_match.index:
    if len(df_match.at[i,'Assay Match']) > 0:
        for match in df_match.at[i,'Assay Match']:
            rmi_name.append(df_match.at[i,'RMI Name'])
            assay_match_name.append(match)
    else:
        rmi_name.append(df_match.at[i,'RMI Name'])
        assay_match_name.append(None)


df_match_unpack = pd.DataFrame(list(zip(rmi_name, assay_match_name)), columns = ['RMI Name', 'Assay Name NN']) 

In [19]:
df_match_unpack[pd.isnull(df_match_unpack['Assay Name NN'])]

,RMI Name,Assay Name NN
262,Anderlues,None
655,Bibiyana,None
891,Bukhara-Khiva,None
1112,Cazaux,None
1124,Champotran,None
...,...,...
7073,Yablunivske,None
7074,Yadana,None
7095,Yapacani,None
7289,Zawtika,None


#### Merge the Assay information into the RMI Name

In [20]:
df_full_match = df_match_unpack.merge(assay_df, how='left', on='Assay Name NN')

In [22]:
#df_full_match[pd.isnull(df_full_match['Assay Name NN'])]

### Merge with Field Info From Runs

In [23]:
# opgee_output_path = '/Users/atecza/Library/CloudStorage/OneDrive-RMI/Documents/ClimateTRACE Phase 4/csv/db_exports/trace_main_outputs.csv'
# opgee_outpus = pd.read_csv(opgee_output_path)
# opgee_outputs_2 = opgee_outpus.loc[:,['Field_name','Region','API_gravity']] #'Gas_composition_H2S'
# opgee_outputs_2.rename(columns={'Year':'RMI Year','Region':'RMI Region','API_gravity':'RMI API'}, inplace=True)
# opgee_outputs_group = opgee_outputs_2.groupby('Field_name').first().reset_index()
# opgee_assay_full = df_full_match.merge(opgee_outputs_group, how = 'left', left_on ='RMI Name', right_on='Field_name')

In [23]:
opgee_assay_full = df_full_match.merge(field_names_df, how = 'left', left_on ='RMI Name', right_on='Field_name')

In [25]:
#opgee_assay_full[pd.isnull(opgee_assay_full['Assay Name NN'])].loc[:,'Field_location_Country'].unique()

In [26]:
opgee_assay_full.drop(columns=['Field_name','Region_x','Country'], inplace=True)

In [33]:
#opgee_assay_full

In [34]:
opgee_assay_full['API Difference'] = abs(opgee_assay_full['API'] - opgee_assay_full['API_gravity'])

In [36]:
#opgee_assay_full[pd.isnull(opgee_assay_full['API Difference'])]

In [42]:
series_list = []
df_group = opgee_assay_full.groupby('RMI Name')

for name_of_group, contents_of_group in df_group:
    
    df1 = df_group.get_group(name_of_group)
    
    #If any of the assays in the RMI Field group have an NA, use API Difference
    if df1['Year'].isnull().any():
        
        #If there is an API Difference
        if df1['API Difference'].notnull().all():
        
            series_list.append(df1.loc[df1['API Difference'].idxmin()])
        
        else:
            #Take the first row
            series_list.append(df1.iloc[0,:])
    
    else:
        series_list.append(df1.loc[df1['Year'].idxmax()])

df_g = pd.concat(series_list, axis=1).T

In [43]:
print(opgee_assay_full.groupby('RMI Name').ngroups)
print(len(df_g))

585
585


In [44]:
df_g

,RMI Name,Assay Name NN,Assay Name,Assay Number,Throughput,Sulphur,API,Year,Field_location_Country,Region_y,US_state,API_gravity,API Difference
1,Abana,Kole_Solomon,Kole_Solomon 2014,NaN,100028.976072,0.347871,30.832788,2014,Cameroon,West Africa,Not US,29.0,1.832788
9,Abqaiq,Arab Light_Solomon,Arab Light_Solomon 2016,NaN,100037.209515,1.896356,34.515023,2016,Saudi Arabia,Middle East,Not US,37.0,2.484977
22,Abu El Gharadiq,Qarun_Solomon,Qarun_Solomon 2018,NaN,100023.445278,0.297064,35.515013,2018,Egypt,North Africa,Not US,35.0,0.515013
30,Abu Sa fah,Arab Heavy_Solomon,Arab Heavy_Solomon 2008,NaN,100043.156183,2.832999,28.525567,2008,Saudi Arabia,Middle East,Not US,28.6,0.074433
39,Abushar,Iran Ardeshir_COA,Iran Ardeshir_COA,98,100398.194653,2.547449,26.9,<NA>,Iran,Middle East,Not US,26.0,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7824,Zhanazhol,Tengiz_Solomon,Tengiz_Solomon 2007,NaN,100037.158812,0.46393,50.477739,2007,Kazakhstan,Central Asia,Not US,49.3,1.177739
7828,Zinia,Angola Kuito_Chevron,Angola Kuito_Chevron,36,96240.387983,0.871157,22.05,<NA>,Angola,West Africa,Not US,20.0,2.05
7863,Zohr,Western Desert_Solomon,Western Desert_Solomon 2017,NaN,100033.685632,0.325706,41.958891,2017,Egypt,North Africa,Not US,50.0,8.041109
7870,Zubair,Kirkuk_Solomon,Kirkuk_Solomon 2015,NaN,100042.08128,2.25189,34.901123,2015,Iraq,Middle East,Not US,35.0,0.098877


In [45]:
df_g.to_excel('/Users/atecza/Library/CloudStorage/OneDrive-RMI/Documents/OCI Phase 2/Midstream/Assay_rematch/assay_match_df.xlsx', index=False)


### Old Code/Sandbox

In [315]:
assay_df['API Diff Normalized'] = abs((assay_df['API'] - assay_df['API'].mean())/(assay_df['API'].std()))
#assay_df['API Normalized'] = abs((assay_df['API'] - assay_df['API'].mean())/(assay_df['API'].std()))

In [109]:
def closest_val(lst, K):
    closest_val_list = []
    dif_list = [abs(x-K) for x in lst]
    s = min(dif_list)

    i_list = [i for i, x in enumerate(dif_list) if x == s]

    for index in i_list:
        closest_val_list.append(lst[index])
    
    return closest_val_list